FinBERT testing with named entity recognition

In [1]:
!pip install -q transformers

In [2]:
!pip install -q newspaper3k pandas numpy

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 22.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 6.5 MB/s eta 0:00:00


In [3]:
!pip install -q lxml-html-clean

In [4]:
import pandas as pd
import numpy as np
import torch

In [5]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())

False
0


In [6]:
from transformers import pipeline

pipe = pipeline("sentiment-analysis", model="ProsusAI/finbert", device=0 if torch.cuda.is_available() else -1)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [7]:
test_data = ["Adherents of the Dogs strategy assume that the highest yielding Dow stocks are unfairly and temporarily depressed. That presumably means they'll hold up better than the S&P 500. And that seems to be the case in 2022. Adding the Dogs stocks' 4% average dividend yield coming into 2022 plus their average 1.8% drop still left investors with a positive return of more than 2% on the stocks. Additionally, half of last year's Dogs, including International Business Machines (IBM), Chevron (CVX), Merck (MRK), Amgen (AMGN), and Coca-Cola (KO) posted gains in 2022. Chevron alone gained 53% on top of yielding 4.6%.",
             "The biggest dog of them all for 2023 is fallen communication services firm Verizon (VZ). The company, which has seen its stock drop 24% in 2022, is now yielding more than any other Dow stock: 6.5%. It's important to note, though, there's a reason Verizon's stock is shrinking. Verizon's profit is seen falling nearly 4% in 2022 and nearly 3% in 2023.",
             "the most average stock is stock ABC, which did not change at all"]

In [8]:
result = pipe(test_data)
print(result)

[{'label': 'positive', 'score': 0.8819694519042969}, {'label': 'negative', 'score': 0.9748380184173584}, {'label': 'neutral', 'score': 0.9416248202323914}]


spacy

In [9]:
!pip install -q spacy

In [10]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [11]:
txt = "Adherents of the Dogs strategy assume that the highest yielding Dow stocks are unfairly and temporarily depressed. That presumably means they'll hold up better than the S&P 500. And that seems to be the case in 2022. Adding the Dogs stocks' 4% average dividend yield coming into 2022 plus their average 1.8% drop still left investors with a positive return of more than 2% on the stocks. Additionally, half of last year's Dogs, including International Business Machines (IBM), Chevron (CVX), Merck (MRK), Amgen (AMGN), and Coca-Cola (KO) posted gains in 2022. Chevron alone gained 53% on top of yielding 4.6%."
doc = nlp(txt)

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Dow 64 67 ORG
2022 211 215 DATE
4% 241 243 PERCENT
2022 279 283 DATE
1.8% 303 307 PERCENT
more than 2% 360 372 PERCENT
half of last year's 402 421 DATE
International Business Machines 438 469 ORG
IBM 471 474 ORG
Chevron 477 484 ORG
CVX 486 489 ORG
Merck 492 497 ORG
Amgen 505 510 ORG
AMGN 512 516 ORG
Coca-Cola 523 532 ORG
2022 554 558 DATE
Chevron 560 567 ORG
53% 581 584 PERCENT
4.6% 604 608 PERCENT


Upload local files for comparison

In [12]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load FinBERT model and tokenizer once
model_name = "ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)



BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [11]:
from newspaper import Article

In [12]:
!ls

sample_data


In [13]:
from google.colab import files
uploaded = files.upload()

Saving AAPL_main.csv to AAPL_main.csv
Saving AAPL_relation.csv to AAPL_relation.csv


In [14]:
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

User uploaded file "AAPL_main.csv" with length 5386490 bytes
User uploaded file "AAPL_relation.csv" with length 1017748 bytes


In [15]:
!ls

AAPL_main.csv  AAPL_relation.csv  sample_data


In [16]:
def get_news_text(url):
    article = Article(url)
    try:
        article.download()
        article.parse()
        return article.text
    except Exception as error:
        return error

In [17]:
def split_text_by_ticker_aliases(text, nlp, ticker_aliases):
    """
    Splits the text into sections based on the presence of ticker names or their aliases,
    as well as a category for text not related to any ticker.
    """
    split_text = {ticker: list([]) for ticker in ticker_aliases.keys()}
    split_text["none"] = list([])  # For text unrelated to any ticker

    # alias_to_ticker = {}
    # for ticker, aliases in ticker_aliases.items():
    #     for alias in aliases:
    #         alias_to_ticker[alias.lower()] = ticker  # Use lowercase for matching

    doc = nlp(text)

    sentences = list(doc.sents)

    for sentence in sentences:
        associated = False

        # Check for ticker matches in named entities
        for ent in sentence.ents:
            alias = ent.text.lower()
            for ticker, aliases in ticker_aliases.items():
                for aliase in aliases:
                    if alias in aliase.lower() or aliase.lower() in alias:
                        split_text[ticker].append(sentence.text)
                        associated = True
                        break


            # if alias in alias_to_ticker:
            #     ticker = alias_to_ticker[alias]
            #     split_text[ticker] += sentence.text + " "
            #     associated = True

        if not associated:
            split_text["none"].append(sentence.text)

    return split_text

In [18]:
ticker_aliases = {
    "AAPL": ("AAPL", "Apple")
}

In [19]:
# example_apple = get_news_text("https://www.fool.com/investing/2024/10/30/10-companies-trillion-dollar-market-cap-1-buy-now/?source=iedfolrf0000001")
# print(example_apple)

In [37]:
splited = split_text_by_ticker_aliases(example_apple, nlp, ticker_aliases)
for k, v in splited.items():
    print(k)
    print(v)

AAPL
["Not accounting for inflationary changes over time (sorry, I'm not including the Dutch East India Company), there have been 10 public companies that have reached the trillion-dollar market cap plateau, of which nine can be bought and sold on U.S. exchanges:\n\nApple AAPL 1.02% )\n\n", 'Some investors might be attracted to Apple given its sizable cash pile, more than $700 billion in share repurchases since 2013, and its No. 1 market share in domestic smartphones.', "To add, CEO Tim Cook is overseeing a shift in Apple's operating model to one focused on higher-margin subscription services.\n\n", "But a deeper dive into Apple's revenue breakdown shows that sales of its physical products, including iPhone, Mac, and iPad, have notably weakened over the last two years.", 'Apple has commanded a premium valuation because of its consistent growth.', "However, with its growth engine stalling, Apple's forward price-to-earnings (P/E) ratio of 31 sticks out like a sore thumb.\n\n"]
none
["One

In [34]:
print(pipe("Some investors might be attracted to Apple given its sizable cash pile, more than $700 billion in share repurchases since 2013, and its No. 1 market share in domestic smartphones. To add, CEO Tim Cook is overseeing a shift in Apple's operating model to one focused on higher-margin subscription services."))

[{'label': 'positive', 'score': 0.754537045955658}]


In [20]:
def combine_dataframes_with_time_range(main_df, relation_df, start_time, end_time):
    start_time = pd.to_datetime(start_time)
    end_time = pd.to_datetime(end_time)

    main_df['publish_time'] = pd.to_datetime(main_df['publish_time'])
    relation_df['time'] = pd.to_datetime(relation_df['time'])
    filtered_rel = relation_df[relation_df['ticker'] == relation_df['source_ticker']]

    # Merge the two dataframes on `id` and `news_id`
    merged_df = main_df.merge(filtered_rel, left_on='id', right_on='news_id', how='inner')

    filtered_df = merged_df

    time_filtered_df = filtered_df[(filtered_df['publish_time'] >= start_time) & (filtered_df['publish_time'] <= end_time)]

    time_filtered_df = time_filtered_df.rename(columns={'ticker_x': 'ticker'})

    result_df = time_filtered_df[['id', 'publish_time', 'article_url', 'sentiment', 'ticker']]

    return result_df

In [ ]:
# def combine_dataframes_with_time_range(main_df, relation_df, start_time, end_time):
#     start_time = pd.to_datetime(start_time)
#     end_time = pd.to_datetime(end_time)

#     main_df['publish_time'] = pd.to_datetime(main_df['publish_time'])
#     relation_df['time'] = pd.to_datetime(relation_df['time'])

#     relation_df_grouped = relation_df.groupby('news_id')['sentiment'].agg(lambda x: x.mode()[0]).reset_index()

#     merged_df = main_df.merge(relation_df_grouped, left_on='id', right_on='news_id', how='inner')

#     time_filtered_df = merged_df[(merged_df['publish_time'] >= start_time) & (merged_df['publish_time'] <= end_time)]

#     result_df = time_filtered_df[['id', 'publish_time', 'article_url', 'sentiment']]

#     return result_df

In [ ]:
# def compare_sentiments_finbert(merged_df, model, tokenizer, truncate_or_split="truncate"):
#     """
#     Compare sentiment labels from FinBERT predictions with existing sentiment labels,
#     handling text sequences longer than 512 tokens.
#     """
#     label_map = {0: "negative", 1: "neutral", 2: "positive"}

#     sentiment_matrix = {
#         "positive": {"positive": 0, "neutral": 0, "negative": 0},
#         "neutral": {"positive": 0, "neutral": 0, "negative": 0},
#         "negative": {"positive": 0, "neutral": 0, "negative": 0},
#     }

#     i = 0

#     for row in merged_df.itertuples():
#         if i % 10 == 0:
#             print(f"Processing: {i}")

#         url = row.article_url
#         text = get_news_text(url)

#         if text != "Error" and isinstance(text, str):
#             if truncate_or_split == "split":
#                 tokens = tokenizer(text, truncation=False, add_special_tokens=True)["input_ids"]

#                 if len(tokens) > 512:
#                     # Split text into chunks of approximately 512 tokens
#                     chunk_size = 512
#                     chunks = [text[start:start + chunk_size] for start in range(0, len(text), chunk_size)]

#                     # Get predictions for each chunk
#                     chunk_predictions = []
#                     for chunk in chunks:
#                         inputs = tokenizer(chunk, return_tensors="pt", truncation=True, max_length=512)
#                         outputs = model(**inputs)
#                         prediction = torch.argmax(outputs.logits, dim=1).item()
#                         chunk_predictions.append(label_map[prediction])

#                     # Use the most frequent label
#                     predicted = max(set(chunk_predictions), key=chunk_predictions.count)
#                 else:
#                     inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
#                     outputs = model(**inputs)
#                     prediction = torch.argmax(outputs.logits, dim=1).item()
#                     predicted = label_map[prediction]
#             else:
#                 inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
#                 outputs = model(**inputs)
#                 prediction = torch.argmax(outputs.logits, dim=1).item()
#                 predicted = label_map[prediction]

#             old = row.sentiment

#             # Increment the corresponding matrix cell
#             if old in sentiment_matrix and predicted in sentiment_matrix:
#                 sentiment_matrix[predicted][old] += 1

#         i += 1

#     result_matrix = pd.DataFrame(sentiment_matrix).T

#     return result_matrix

def compare_sentiments_finbert(merged_df, model, tokenizer, truncate_or_split="truncate"):
    """
    Compare sentiment labels from FinBERT predictions with existing sentiment labels,
    handling text sequences longer than 512 tokens and ensuring the model and inputs are moved to the GPU.
    """
    label_map = {0: "negative", 1: "neutral", 2: "positive"}

    sentiment_matrix = {
        "positive": {"positive": 0, "neutral": 0, "negative": 0},
        "neutral": {"positive": 0, "neutral": 0, "negative": 0},
        "negative": {"positive": 0, "neutral": 0, "negative": 0},
    }


    i = 0

    for row in merged_df.itertuples():
        if i % 10 == 0:
            print(f"Processing: {i}")

        url = row.article_url
        text = get_news_text(url)

        if text != "Error" and isinstance(text, str):
            if truncate_or_split == "split":
                tokens = tokenizer(text, truncation=False, add_special_tokens=True)["input_ids"]

                if len(tokens) > 512:
                    chunk_size = 512
                    chunks = [text[start:start + chunk_size] for start in range(0, len(text), chunk_size)]

                    chunk_predictions = []
                    for chunk in chunks:
                        inputs = tokenizer(chunk, return_tensors="pt", truncation=True, max_length=512)
                        inputs = {key: val.to(device) for key, val in inputs.items()}  # Move inputs to GPU
                        outputs = model(**inputs)
                        prediction = torch.argmax(outputs.logits, dim=1).item()
                        chunk_predictions.append(label_map[prediction])

                    # Use the most frequent label
                    predicted = max(set(chunk_predictions), key=chunk_predictions.count)
                else:
                    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
                    inputs = {key: val.to(device) for key, val in inputs.items()}  # Move inputs to GPU
                    outputs = model(**inputs)
                    prediction = torch.argmax(outputs.logits, dim=1).item()
                    predicted = label_map[prediction]
            else:
                inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
                inputs = {key: val.to(device) for key, val in inputs.items()}  # Move inputs to GPU
                outputs = model(**inputs)
                prediction = torch.argmax(outputs.logits, dim=1).item()
                predicted = label_map[prediction]

            old = row.sentiment

            if old in sentiment_matrix and predicted in sentiment_matrix:
                sentiment_matrix[predicted][old] += 1

        i += 1

    result_matrix = pd.DataFrame(sentiment_matrix).T
    return result_matrix


In [ ]:
# def compare_sentiments_finbert_pipeline(merged_df, pipe, truncate_or_split="truncate"):
#     """
#     Compare sentiment labels from FinBERT predictions with existing sentiment labels,
#     handling text sequences longer than 512 tokens using the transformers pipeline.
#     """
#     sentiment_matrix = {
#         "positive": {"positive": 0, "neutral": 0, "negative": 0},
#         "neutral": {"positive": 0, "neutral": 0, "negative": 0},
#         "negative": {"positive": 0, "neutral": 0, "negative": 0},
#     }

#     i = 0

#     for row in merged_df.itertuples():
#         if i % 10 == 0:
#             print(f"Processing: {i}")

#         url = row.article_url
#         text = get_news_text(url)

#         if text != "Error" and isinstance(text, str):
#             if truncate_or_split == "split":
#                 tokens = pipe.tokenizer(text, truncation=False, add_special_tokens=True)["input_ids"]

#                 if len(tokens) > 512:
#                     chunk_size = 512
#                     chunks = [text[start:start + chunk_size] for start in range(0, len(text), chunk_size)]

#                     # Get predictions for each chunk
#                     chunk_predictions = []
#                     for chunk in chunks:
#                         result = pipe(chunk)
#                         chunk_predictions.append(result[0]['label'])

#                     # Use the most frequent label
#                     predicted = max(set(chunk_predictions), key=chunk_predictions.count)
#                 else:
#                     result = pipe(text)
#                     predicted = result[0]['label']
#             else:
#                 result = pipe(text, truncation=True)
#                 predicted = result[0]['label']

#             old = row.sentiment

#             if old in sentiment_matrix and predicted in sentiment_matrix:
#                 sentiment_matrix[predicted][old] += 1

#         i += 1

#     result_matrix = pd.DataFrame(sentiment_matrix).T  # Transpose for correct layout

#     return result_matrix


In [22]:
def compare_sentiments_finbert_pipeline(merged_df, pipe, truncate_or_split="truncate"):
    """
    Compare sentiment labels from FinBERT predictions with existing sentiment labels,
    handling text sequences longer than 512 tokens using the transformers pipeline.
    """
    sentiment_matrix = {
        "positive": {"positive": 0, "neutral": 0, "negative": 0},
        "neutral": {"positive": 0, "neutral": 0, "negative": 0},
        "negative": {"positive": 0, "neutral": 0, "negative": 0},
    }

    i = 0

    for row in merged_df.itertuples():
        if i % 10 == 0:
            print(f"Processing: {i}")

        url = row.article_url
        text = get_news_text(url)

        if text != "Error" and isinstance(text, str):
            if truncate_or_split == "split":
                tokens = pipe.tokenizer(text, truncation=False, add_special_tokens=True)["input_ids"]

                if len(tokens) > 512:
                    chunk_size = 512
                    chunks = [text[start:start + chunk_size] for start in range(0, len(text), chunk_size)]

                    # Get predictions for each chunk
                    chunk_predictions = []
                    for chunk in chunks:
                        result = pipe(chunk)
                        chunk_predictions.append(result[0]['label'])

                    # Use the most frequent label
                    predicted = max(set(chunk_predictions), key=chunk_predictions.count)
                else:
                    result = pipe(text)
                    predicted = result[0]['label']
            else:
                result = pipe(text, truncation=True)
                predicted = result[0]['label']

            old = row.sentiment

            if old in sentiment_matrix and predicted in sentiment_matrix:
                sentiment_matrix[predicted][old] += 1

        i += 1

    result_matrix = pd.DataFrame(sentiment_matrix).T

    return result_matrix

In [ ]:
def analyze_sentiments_finbert_pipeline(merged_df, pipe, truncate_or_split="truncate"):
    """
    Perform sentiment analysis on financial news articles using FinBERT,
    processing text sequences longer than 512 tokens as needed.
    """
    # Initialize an empty list to store results
    result_data = []

    for i, row in enumerate(merged_df.itertuples(), start=1):
        if i % 10 == 0:
            print(f"Processing article {i}/{len(merged_df)}")

        url = row.article_url
        text = get_news_text(url)

        if text != "Error" and isinstance(text, str):
            if truncate_or_split == "split":
                tokens = pipe.tokenizer(text, truncation=False, add_special_tokens=True)["input_ids"]

                if len(tokens) > 512:
                    chunk_size = 512
                    chunks = [text[start:start + chunk_size] for start in range(0, len(text), chunk_size)]

                    # Get predictions for each chunk
                    chunk_predictions = []
                    for chunk in chunks:
                        result = pipe(chunk)
                        chunk_predictions.append(result[0]['label'])

                    # Use the most frequent label
                    predicted_sentiment = max(set(chunk_predictions), key=chunk_predictions.count)
                else:
                    result = pipe(text)
                    predicted_sentiment = result[0]['label']
            else:
                result = pipe(text, truncation=True)
                predicted_sentiment = result[0]['label']

            result_data.append({
                "ticker": row.ticker,
                "sentiment": predicted_sentiment,
                "news_id": row.id,
                "source_ticker": row.ticker,
                "time": row.publish_time
            })

    result_df = pd.DataFrame(result_data, columns=['ticker', 'sentiment', 'news_id', 'source_ticker', 'time'])

    return result_df

In [23]:
def add_explanations_to_matrix(result_matrix):
    row_explanations = [
        "Predicted as positive",
        "Predicted as neutral",
        "Predicted as negative",
    ]
    result_matrix.insert(0, "Explanation", row_explanations)


    explanation_row = {
        "Explanation": "Row description",
        "positive": "Labeled positive",
        "neutral": "Labeled neutral",
        "negative": "Labeled negative",
    }

    explanation_df = pd.DataFrame([explanation_row])
    result_matrix = pd.concat([result_matrix, explanation_df], ignore_index=True)

    return result_matrix

In [29]:
df_news = pd.read_csv('AAPL_main.csv')
df_rela = pd.read_csv('AAPL_relation.csv')

In [30]:
merged_df = combine_dataframes_with_time_range(df_news, df_rela, "2024-07-01", "2024-09-01")

In [31]:
print(merged_df)

                                                    id        publish_time  \
245  f9b58febeb4565a6dd607e3216decb281b237a1e76b9e9... 2024-08-31 11:15:00   
246  9cc8d83bae696e5f0565a1a0391596b06171e5ed3e8402... 2024-08-31 10:24:00   
247  f83a6765bf7b69e13cdef5999c8e5c99a26598acdce856... 2024-08-31 10:10:00   
248  42cf4288fdeddba7d0846f4bdcdb97946c96f0849c11d0... 2024-08-31 07:01:00   
249  d3e51924ac19db5ffb2f24d1621a9a213f30f9ed3aef5c... 2024-08-30 14:14:29   
..                                                 ...                 ...   
658  1fcac93fed0e133da96bfb1c8953aac2f52188cd442308... 2024-07-02 23:30:49   
659  860ffff9a7f8e6ac55b2ddd19bd169ecd1a9f503e9f1dc... 2024-07-02 21:05:40   
660  4d843941162f622729de3d2276012094d587ca8dfb916f... 2024-07-02 20:15:43   
661  1d7b72c113d7c86046e8832efab111ddbc673a08a41547... 2024-07-02 20:14:53   
662  61d80aa431470bc0a67ae461d70964cc7827fa8a37eefd... 2024-07-02 19:16:19   

                                           article_url sentimen

In [ ]:
# result_matrix = compare_sentiments_finbert(merged_df, model, tokenizer, truncate_or_split="truncate")

In [32]:
result_matrix = compare_sentiments_finbert_pipeline(merged_df, pipe, nlp, truncate_or_split="truncate")

Processing: 0
Processing: 10
Processing: 20
Processing: 30
Processing: 40
Processing: 50
Processing: 60
Processing: 70
Processing: 80
Processing: 90
Processing: 100
Processing: 110
Processing: 120
Processing: 130
Processing: 140
Processing: 150
Processing: 160
Processing: 170
Processing: 180
Processing: 190
Processing: 200
Processing: 210
Processing: 220
Processing: 230
Processing: 240
Processing: 250
Processing: 260
Processing: 270
Processing: 280
Processing: 290
Processing: 300
Processing: 310
Processing: 320
Processing: 330
Processing: 340
Processing: 350
Processing: 360
Processing: 370
Processing: 380
Processing: 390
Processing: 400
Processing: 410


Using spaCy NER to split text into categories by their check their sentiments, performance improved significantly

In [33]:
print(add_explanations_to_matrix(result_matrix))

             Explanation          positive          neutral          negative
0  Predicted as positive               117               35                 6
1   Predicted as neutral                90               68                11
2  Predicted as negative                17               17                24
3        Row description  Labeled positive  Labeled neutral  Labeled negative


In [ ]:
# result_df = analyze_sentiments_finbert_pipeline(merged_df, pipe, truncate_or_split="split")

Token indices sequence length is longer than the specified maximum sequence length for this model (807 > 512). Running this sequence through the model will result in indexing errors
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Processing article 10/266
Processing article 20/266
Processing article 30/266
Processing article 40/266
Processing article 50/266
Processing article 60/266
Processing article 70/266
Processing article 80/266
Processing article 90/266
Processing article 100/266
Processing article 110/266
Processing article 120/266
Processing article 130/266
Processing article 140/266
Processing article 150/266
Processing article 160/266
Processing article 170/266
Processing article 180/266
Processing article 190/266
Processing article 200/266
Processing article 210/266
Processing article 220/266
Processing article 230/266
Processing article 240/266
Processing article 250/266
Processing article 260/266


In [ ]:
print(result_df)

    ticker sentiment                                            news_id  \
0     AAPL   neutral  98c6ca14cab03a3bd104600a398277c200673939a9ef65...   
1     AAPL   neutral  c6ad5b53e41888dcaad92a4fa2af4f82b8f7d3649cd883...   
2     AAPL   neutral  5c9942bce64846d690ff4a1593f767e2b7712c6ad1ff8f...   
3     AAPL   neutral  c1f4e581e6ad7955af2a52c2f98fd20d63597fae248d7b...   
4     AAPL  negative  0e2c7c16e5f2b68814f368a981536eeb048c946e68af80...   
..     ...       ...                                                ...   
213   AAPL  negative  9157017d50628fbd2c24329b6f42207f23a58271cb2ef6...   
214   AAPL  positive  3ce64356f2d9ee55998edb22e8b44897fb279d3a3b88d7...   
215   AAPL  negative  6c0bc65d6a4675fecebd1e3838124acb9c992cf4182624...   
216   AAPL  positive  1fcac93fed0e133da96bfb1c8953aac2f52188cd442308...   
217   AAPL   neutral  860ffff9a7f8e6ac55b2ddd19bd169ecd1a9f503e9f1dc...   

    source_ticker                time  
0            AAPL 2024-07-31 18:12:00  
1            AAPL 2

In [ ]:
result_df.to_csv('AAPL_rel_2.csv', index=False)

In [ ]:
files.download('AAPL_rel_2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# result_matrix = add_explanations_to_matrix(result_matrix)
# print(result_matrix)

             Explanation          positive          neutral          negative
0  Predicted as positive               115               60                15
1   Predicted as neutral                38               26                16
2  Predicted as negative                79               32                10
3        Row description  Labeled positive  Labeled neutral  Labeled negative


In [ ]:
result_matrix = compare_sentiments_finbert(merged_df, model, tokenizer, truncate_or_split="split")

Processing: 0
Processing: 10
Processing: 20
Processing: 30
Processing: 40
Processing: 50
Processing: 60
Processing: 70
Processing: 80
Processing: 90
Processing: 100
Processing: 110
Processing: 120
Processing: 130
Processing: 140
Processing: 150
Processing: 160
Processing: 170
Processing: 180
Processing: 190
Processing: 200
Processing: 210
Processing: 220
Processing: 230
Processing: 240
Processing: 250
Processing: 260
Processing: 270
Processing: 280
Processing: 290
Processing: 300
Processing: 310
Processing: 320
Processing: 330
Processing: 340
Processing: 350
Processing: 360
Processing: 370
Processing: 380
Processing: 390
Processing: 400
Processing: 410


In [ ]:
result_matrix = add_explanations_to_matrix(result_matrix)
print(result_matrix)

             Explanation          positive          neutral          negative
0  Predicted as positive               125               63                20
1   Predicted as neutral                24               18                16
2  Predicted as negative                83               40                 4
3        Row description  Labeled positive  Labeled neutral  Labeled negative


In [ ]:
result_matrix = compare_sentiments_finbert_pipeline(merged_df, pipe, truncate_or_split="truncate")

Processing: 0
Processing: 10


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Processing: 20
Processing: 30
Processing: 40
Processing: 50
Processing: 60
Processing: 70
Processing: 80
Processing: 90
Processing: 100
Processing: 110
Processing: 120
Processing: 130
Processing: 140
Processing: 150
Processing: 160
Processing: 170
Processing: 180
Processing: 190
Processing: 200
Processing: 210
Processing: 220
Processing: 230
Processing: 240
Processing: 250
Processing: 260
Processing: 270
Processing: 280
Processing: 290
Processing: 300
Processing: 310
Processing: 320
Processing: 330
Processing: 340
Processing: 350
Processing: 360
Processing: 370
Processing: 380
Processing: 390
Processing: 400
Processing: 410


In [ ]:
result_matrix = add_explanations_to_matrix(result_matrix)
print(result_matrix)

             Explanation          positive          neutral          negative
0  Predicted as positive                75               38                10
1   Predicted as neutral               103               76                14
2  Predicted as negative                36               21                22
3        Row description  Labeled positive  Labeled neutral  Labeled negative


In [ ]:
result_matrix = compare_sentiments_finbert_pipeline(merged_df, pipe, truncate_or_split="split")

Processing: 0


Token indices sequence length is longer than the specified maximum sequence length for this model (1457 > 512). Running this sequence through the model will result in indexing errors


Processing: 10
Processing: 20
Processing: 30
Processing: 40
Processing: 50
Processing: 60
Processing: 70
Processing: 80
Processing: 90
Processing: 100
Processing: 110
Processing: 120
Processing: 130
Processing: 140
Processing: 150
Processing: 160
Processing: 170
Processing: 180
Processing: 190
Processing: 200
Processing: 210
Processing: 220
Processing: 230
Processing: 240
Processing: 250
Processing: 260
Processing: 270
Processing: 280
Processing: 290
Processing: 300
Processing: 310
Processing: 320
Processing: 330
Processing: 340
Processing: 350
Processing: 360
Processing: 370
Processing: 380
Processing: 390
Processing: 400
Processing: 410


In [ ]:
result_matrix = add_explanations_to_matrix(result_matrix)
print(result_matrix)

             Explanation          positive          neutral          negative
0  Predicted as positive                90               49                 6
1   Predicted as neutral               102               71                23
2  Predicted as negative                22               15                17
3        Row description  Labeled positive  Labeled neutral  Labeled negative
